## The MSM forward algorithm.

This section simulates what happens in the fitting method of the Markov Switching Multifractal (MSM) model. Differently from standard HMMs, the MSM cannot be fit using the EM algorithm, and requires numerical methods to be applied to the likelihood function instead. 

We start by importing the necessary packages and by creating a test series.

In [1]:
import numpy as np
import jax.numpy as jnp

ts_test = jnp.array(np.random.normal(50, 10, 10))

The model requires the following hyperparameters:
 - `num_latent`: how many volatility components, integer.
 - `marg_prob_mass`: the probability mass of the marginal distribution of the latent states, needs to sum to 1.

In [2]:
n_latent = 3
marg_prob = jnp.full(2, 0.5)

For this simulation we also use example parameters. In the `MSM` class the parameters are instead found starting from an intial guess using the `MSM.fit()` method.

By assumption, all the parameters need to be positive, and have further individual constrains:

- `marg_support`: the support of the marginal probability mass defined in the parameters. It needs to have unity expectation. In the symmetric binomial case, this can be enforced by specifying one value $m_0$, and having the second value be $2- m_0$.

- `unconditional_term`: the unconditional distribution of the model, a positive double.

- `arrival_gdistance`: the geometric distance between the Poisson arrivals of each latent volatility component, a positive double.

- `hf_arrival`: the highest poisson arrival probability (i.e. the proability of state switch of the highest frequency component)

In [3]:
m0 = 0.45677
m_support = jnp.array([2-m0, m0])
unconditional_term = 1.0
arrival_gdistance = 3.0
hf_arrival = 0.98

As in the `MSM` class, we proceed to initialize the components of the forward algorithm: the ergotic distribution of the model, the emission probabilities (data likelihood) and the transition tensor.

In [4]:
from fractrics._ts_components._HMM._initial_distribution import multiplicative_cascade
from fractrics._ts_components._HMM._data_likelihood import dlk_normal
from fractrics._ts_components._HMM._transition_tensor import poisson_arrival

# simulating what happens in the __init__ of the model: initialization of components
indis_fn = multiplicative_cascade(marg_prob_mass=marg_prob, num_latent=n_latent)
dl_fn = dlk_normal(ts_test)
trtens_fn = poisson_arrival(num_latent=n_latent, marg_prob_mass=marg_prob)

# computing components
latent_states = indis_fn.support(uncond_term=unconditional_term, marg_support=m_support)
ergotic_distr = indis_fn.mass()
data_likelihood = dl_fn.likelihood(latent_states=latent_states)
transition_tensor = trtens_fn.t_tensor(arrival_gdistance=arrival_gdistance, hf_arrival=hf_arrival)

Then the forward class takes them as input and computes the recursion for all datapoints. The forward returns the negative log likelihood of the model, the final posterior distribution of the latent states, and all the intermediate updates.

In [5]:
from fractrics._ts_components._HMM._forward import factor_transition

forward_fn = factor_transition(num_latent=n_latent)
forward_predictive_fn = forward_fn.make_predictive_function(*transition_tensor)
predictive = forward_predictive_fn(prior=ergotic_distr)

nll, posterior, list = forward_fn.update(ergotic_distr, data_likelihood, transition_tensor)